## Setup

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>.output_result { max-width:90% !important; }</style>"))

import warnings
warnings.filterwarnings("ignore")

import numpy as np
np.set_printoptions(precision=3)
np.set_printoptions(linewidth=170)
np.set_printoptions(suppress=True)
np.random.seed(8)

from time import time, sleep
sleep(0.2)

C:\Users\raziel lab\AppData\Local\Temp\ipykernel_30804\1403808273.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


## Imports & Parameters

In [2]:
import pandas as pd

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows', None)
random_state = 8

## Loading the data

In [3]:
refrigerators_1 = pd.read_csv('C:/Users/raziel lab/PycharmProjects/ConsumptionModels/Appliances Algorithm/Refrigerators/refrigerators1.csv', encoding='utf8')
refrigerators_1.shape

(816, 26)

In [4]:
refrigerators_2 = pd.read_csv('C:/Users/raziel lab/PycharmProjects/ConsumptionModels/Appliances Algorithm/Refrigerators/refrigerators2.csv', encoding='utf8')
refrigerators_2.shape

(594, 26)

## Main

In [5]:
refrigerators = pd.concat([refrigerators_1, refrigerators_2])
refrigerators.shape

(1410, 26)

In [6]:
len(refrigerators['אינוונטר'].unique())

1410

In [7]:
refrigerators['שם סטטוס'].unique()

array(['פעיל', 'מאושר\xa0לגריעה', 'נגרע', 'לא\xa0מנוהל'], dtype=object)

In [8]:
refrigerators = refrigerators[(refrigerators['שם סטטוס'] == 'לא מנוהל') | (refrigerators['שם סטטוס'] == 'פעיל')]
refrigerators['שם סטטוס'].unique()

array(['פעיל', 'לא\xa0מנוהל'], dtype=object)

In [9]:
print("Rows Number:",refrigerators.shape[0])

Rows Number: 1353


In [10]:
refrigerators['שם קמפוס'].unique()

array([nan, 'קמפוס\xa0מרקוס', 'מדעי\xa0הבריאות\xa0סורוקה', 'קמפוס\xa0שדה\xa0בוקר', 'קרית\xa0ברגמן', 'פארק\xa0הייטק', "מעונות\xa0ג'", "מעונות\xa0ד'", 'קמפוס\xa0אילת',
       'קמפוס\xa0היאס'], dtype=object)

In [11]:
refrigerators = refrigerators[(refrigerators['שם קמפוס'] == 'קמפוס מרקוס') | (pd.isnull(refrigerators['שם קמפוס']))]
refrigerators['שם קמפוס'].unique()

array([nan, 'קמפוס\xa0מרקוס'], dtype=object)

In [12]:
print("Rows Number:", refrigerators.shape[0])

Rows Number: 847


In [13]:
refrigerators.drop(['אינוונטר','סטטוס','שם סטטוס','תאריך עדכון אחרון','פריט','מחיר כניסה בדולר','הזמנה','סריאלי','מחיר ידני','קוד סימון','תז שואל','פטור מותנה','מנוהל תמיד','קמפוס','שם קמפוס','קומה','חדר','הערות לפריט אומנות'], axis=1, inplace=True)
refrigerators.reset_index(drop=True, inplace=True)
refrigerators.head()

,תאריך קבלה,"מחיר כניסה בש""ח",תיאור האינוונטר,הערות לשורה מההזמנה,מחלקה,שם מחלקה,בניין,שם בניין
0,01/08/1993,"72,017.60",R.M. PHOTONICS .REFRIGERATOR AMNIS CCS 200 Spe...,R.M. PHOTONICS .REFRIGERATOR AMNIS CCS 200,5105,פיסיקה,95,בנין מעבדות למחקרים מתקדמים)מט''מ) - 95
1,10/08/2015,"67,585.36","Innova 42R Refrigerated Incubator Shaker, 230 V",כולל האביזרים המפורטים בהצעת המחיר. = כולל שנת...,5910,מיקרוביולוגיה - מדעי הבריאות,41,"בניין המכון הלאומי לביוטכנולוגיה בנגב, ע""ש אדג..."
2,15/10/1991,"46,001.24",LAVANT-DU PONT .CENTRIFUGE-REFRIGERATING,LAVANT-DU PONT .CENTRIFUGE-REFRIGERATING,5901,דיקן הפקולטה למדעי הבריאות,NaN,NaN
3,04/03/2012,"44,371.00",INCUBATOR REFRIGERATED MAX-Q 6000,INCUBATOR REFRIGERATED MAX-Q 6000,5104,מדעי החיים,NaN,NaN
4,10/08/2015,"43,222.55","Innova 42R Refrigerated Incubator Shaker, 230 V","220 V, 50 Hz .Shelf. Overall Dimensions: 62(W...",5513,ביוטכנולוגיה,41,"בניין המכון הלאומי לביוטכנולוגיה בנגב, ע""ש אדג..."


In [14]:
refrigerators.rename(columns = {'תאריך קבלה':'operating_date',
                                'מחיר כניסה בש"ח':'price',
                                'תיאור האינוונטר':'description',
                                'הערות לשורה מההזמנה':'comment',
                                'מחלקה':'department_number',
                                'שם מחלקה':'department_name',
                                'בניין':'building_number',
                                'שם בניין':'building_name'}, inplace=True)
refrigerators.head()

,operating_date,price,description,comment,department_number,department_name,building_number,building_name
0,01/08/1993,"72,017.60",R.M. PHOTONICS .REFRIGERATOR AMNIS CCS 200 Spe...,R.M. PHOTONICS .REFRIGERATOR AMNIS CCS 200,5105,פיסיקה,95,בנין מעבדות למחקרים מתקדמים)מט''מ) - 95
1,10/08/2015,"67,585.36","Innova 42R Refrigerated Incubator Shaker, 230 V",כולל האביזרים המפורטים בהצעת המחיר. = כולל שנת...,5910,מיקרוביולוגיה - מדעי הבריאות,41,"בניין המכון הלאומי לביוטכנולוגיה בנגב, ע""ש אדג..."
2,15/10/1991,"46,001.24",LAVANT-DU PONT .CENTRIFUGE-REFRIGERATING,LAVANT-DU PONT .CENTRIFUGE-REFRIGERATING,5901,דיקן הפקולטה למדעי הבריאות,NaN,NaN
3,04/03/2012,"44,371.00",INCUBATOR REFRIGERATED MAX-Q 6000,INCUBATOR REFRIGERATED MAX-Q 6000,5104,מדעי החיים,NaN,NaN
4,10/08/2015,"43,222.55","Innova 42R Refrigerated Incubator Shaker, 230 V","220 V, 50 Hz .Shelf. Overall Dimensions: 62(W...",5513,ביוטכנולוגיה,41,"בניין המכון הלאומי לביוטכנולוגיה בנגב, ע""ש אדג..."


In [15]:
refrigerators.shape

(847, 8)

In [16]:
domestic_df = pd.DataFrame(columns=refrigerators.columns)
industrial_df = pd.DataFrame(columns=refrigerators.columns)
other_df = pd.DataFrame(columns=refrigerators.columns)

In [17]:
description_col = refrigerators.columns.get_loc('description')

for i in range (refrigerators['description'].shape[0]):
    
    if 'CIRCULATOR' in refrigerators.iloc[i,description_col]:
        refrigerators.iloc[i,description_col] = 'Other'
    if 'Circulator' in refrigerators.iloc[i,description_col]:
        refrigerators.iloc[i,description_col] = 'Other'
    if 'circulator' in refrigerators.iloc[i,description_col]:
        refrigerators.iloc[i,description_col] = 'Other'
    if 'INCUBATOR' in refrigerators.iloc[i,description_col]:
        refrigerators.iloc[i,description_col] = 'Other'
    if 'Incubator' in refrigerators.iloc[i,description_col]:
        refrigerators.iloc[i,description_col] = 'Other'
    if 'incubator' in refrigerators.iloc[i,description_col]:
        refrigerators.iloc[i,description_col] = 'Other'
    if 'CENTRIFUGE' in refrigerators.iloc[i,description_col]:
        refrigerators.iloc[i,description_col] = 'Other'
    if 'Centrifuge' in refrigerators.iloc[i,description_col]:
        refrigerators.iloc[i,description_col] = 'Other'
    if 'centrifuge' in refrigerators.iloc[i,description_col]:
        refrigerators.iloc[i,description_col] = 'Other'
    if 'Digital Precise' in refrigerators.iloc[i,description_col]:
        refrigerators.iloc[i,description_col] = 'Other'
    if 'ALEX RED' in refrigerators.iloc[i,description_col]:
        refrigerators.iloc[i,description_col] = 'Other'
    if 'LABOTAL LTD .REFRIGERATOR' in refrigerators.iloc[i,description_col]:
        refrigerators.iloc[i,description_col] = 'Other'
    if 'Special' in refrigerators.iloc[i,description_col]:
        refrigerators.iloc[i,description_col] = 'Other'  
    if 'INDUSTRIAL' in refrigerators.iloc[i,description_col]:
        refrigerators.iloc[i,description_col] = 'Industrial'
    if 'Industrial' in refrigerators.iloc[i,description_col]:
        refrigerators.iloc[i,description_col] = 'Industrial'
    if 'industrial' in refrigerators.iloc[i,description_col]:
        refrigerators.iloc[i,description_col] = 'Industrial'
    if 'תעשייתי' in refrigerators.iloc[i,description_col]:
        refrigerators.iloc[i,description_col] = 'Industrial'
    if 'אחסון' in refrigerators.iloc[i,description_col]:
        refrigerators.iloc[i,description_col] = 'Industrial'
    if 'IARP' in refrigerators.iloc[i,description_col]:
        refrigerators.iloc[i,description_col] = 'Industrial'
    if 'KIRUR BENNY' in refrigerators.iloc[i,description_col]:
        refrigerators.iloc[i,description_col] = 'Industrial'
    if 'KIRUR FELDMAN' in refrigerators.iloc[i,description_col]:
        refrigerators.iloc[i,description_col] = 'Industrial'
    if 'MONDIAL' in refrigerators.iloc[i,description_col]:
        refrigerators.iloc[i,description_col] = 'Industrial'
    if 'ICE' in refrigerators.iloc[i,description_col]:
        refrigerators.iloc[i,description_col] = 'Industrial'
    if 'AMNIS' in refrigerators.iloc[i,description_col]:
        refrigerators.iloc[i,description_col] = 'Industrial'

In [18]:
other_rows = refrigerators[refrigerators['description'] == 'Other']
industrial_rows = refrigerators[refrigerators['description'] == 'Industrial']

other_df = other_df.append(other_rows, ignore_index = True)
industrial_df = industrial_df.append(industrial_rows, ignore_index = True)

refrigerators.drop(other_rows.index, inplace = True)
refrigerators.drop(industrial_rows.index, inplace = True)

domestic_df = domestic_df.append(refrigerators, ignore_index = True)

other_df.drop(['description', 'comment'], axis=1, inplace = True)
industrial_df.drop(['description', 'comment'], axis=1, inplace = True)
domestic_df.drop(['description', 'comment'], axis=1, inplace = True)

In [19]:
other_without_building = other_df[pd.isnull(other_df['building_number']) == True]
print('Other Refrigerators Without Building:',len(other_without_building))
other_with_building = other_df[pd.isnull(other_df['building_number']) == False]
print('Other Refrigerators With Building:',len(other_with_building))
print()

industrial_without_building = industrial_df[pd.isnull(industrial_df['building_number']) == True]
print('Industrial Refrigerators Without Building:',len(industrial_without_building))
industrial_with_building = industrial_df[pd.isnull(industrial_df['building_number']) == False]
print('Industrial Refrigerators With Building:',len(industrial_with_building))
print()

domestic_without_building = domestic_df[pd.isnull(domestic_df['building_number']) == True]
print('Domestic Refrigerators Without Building:',len(domestic_without_building))
domestic_with_building = domestic_df[pd.isnull(domestic_df['building_number']) == False]
print('Domestic Refrigerators With Building:',len(domestic_with_building))

Other Refrigerators Without Building: 42
Other Refrigerators With Building: 41

Industrial Refrigerators Without Building: 37
Industrial Refrigerators With Building: 43

Domestic Refrigerators Without Building: 561
Domestic Refrigerators With Building: 123


In [20]:
other_without_department = other_df[pd.isnull(other_df['department_name']) == True]
print('Other Refrigerators Without Department:',len(other_without_department))
print()

industrial_without_department = industrial_df[pd.isnull(industrial_df['department_name']) == True]
print('Industrial Refrigerators Without Department:',len(industrial_without_department))
print()

domestic_without_department = domestic_df[pd.isnull(domestic_df['department_name']) == True]
print('Domestic Refrigerators Without Department:',len(domestic_without_department))

Other Refrigerators Without Department: 0

Industrial Refrigerators Without Department: 0

Domestic Refrigerators Without Department: 0


In [21]:
print('Unique Departments in Other df:', other_with_building['department_name'].unique().shape[0])
print()

print('Unique Departments in Industrial df:', industrial_with_building['department_name'].unique().shape[0])
print()

print('Unique Departments in Domestic df:', domestic_with_building['department_name'].unique().shape[0])
print()

Unique Departments in Other df: 14

Unique Departments in Industrial df: 17

Unique Departments in Domestic df: 54



In [22]:
other_div_percentage = other_with_building.groupby(['department_name','building_number']).size().reset_index(name='count')
other_div_percentage_sum = other_div_percentage.groupby(['department_name']).sum()
other_div_percentage_count = other_div_percentage.merge(other_div_percentage_sum, how='left', on='department_name')
other_div_percentage_count['percentage'] = other_div_percentage_count['count_x'].divide(other_div_percentage_count['count_y'], axis=0)
other_div_percentage.drop(['count'], axis=1, inplace=True)
other_div_percentage['percentage'] = other_div_percentage_count['percentage'].values

industrial_div_percentage = industrial_with_building.groupby(['department_name','building_number']).size().reset_index(name='count')
industrial_div_percentage_sum = industrial_div_percentage.groupby(['department_name']).sum()
industrial_div_percentage_count = industrial_div_percentage.merge(industrial_div_percentage_sum, how='left', on='department_name')
industrial_div_percentage_count['percentage'] = industrial_div_percentage_count['count_x'].divide(industrial_div_percentage_count['count_y'], axis=0)
industrial_div_percentage.drop(['count'], axis=1, inplace=True)
industrial_div_percentage['percentage'] = industrial_div_percentage_count['percentage'].values

domestic_div_percentage = domestic_with_building.groupby(['department_name','building_number']).size().reset_index(name='count')
domestic_div_percentage_sum = domestic_div_percentage.groupby(['department_name']).sum()
domestic_div_percentage_count = domestic_div_percentage.merge(domestic_div_percentage_sum, how='left', on='department_name')
domestic_div_percentage_count['percentage'] = domestic_div_percentage_count['count_x'].divide(domestic_div_percentage_count['count_y'], axis=0)
domestic_div_percentage.drop(['count'], axis=1, inplace=True)
domestic_div_percentage['percentage'] = domestic_div_percentage_count['percentage'].values

In [23]:
for i in range (other_without_building['department_name'].unique().shape[0]):
    rows_for_change = other_without_building[other_without_building['department_name'] == other_without_building['department_name'].unique()[i]]
    percentage_for_insert = other_div_percentage[other_div_percentage['department_name'] == other_without_building['department_name'].unique()[i]]
    num_of_different_buildings = len(percentage_for_insert)
    interval1 = 0
    interval2 = 0
    for j in range (num_of_different_buildings):
        interval2 = interval2 + round(rows_for_change.shape[0]*percentage_for_insert.iloc[j,percentage_for_insert.columns.get_loc('percentage')])
        if j == (num_of_different_buildings-1):
            interval2 = len(rows_for_change)
        rows_for_change.iloc[interval1:interval2,rows_for_change.columns.get_loc('building_number')] = percentage_for_insert.iloc[j,percentage_for_insert.columns.get_loc('building_number')]
        interval1 = interval1 + round(rows_for_change.shape[0]*percentage_for_insert.iloc[j,percentage_for_insert.columns.get_loc('percentage')])
        other_without_building[other_without_building['department_name'] == other_without_building['department_name'].unique()[i]] = rows_for_change
        
for i in range (industrial_without_building['department_name'].unique().shape[0]):
    rows_for_change = industrial_without_building[industrial_without_building['department_name'] == industrial_without_building['department_name'].unique()[i]]
    percentage_for_insert = industrial_div_percentage[industrial_div_percentage['department_name'] == industrial_without_building['department_name'].unique()[i]]
    num_of_different_buildings = len(percentage_for_insert)
    interval1 = 0
    interval2 = 0
    for j in range (num_of_different_buildings):
        interval2 = interval2 + round(rows_for_change.shape[0]*percentage_for_insert.iloc[j,percentage_for_insert.columns.get_loc('percentage')])
        if j == (num_of_different_buildings-1):
            interval2 = len(rows_for_change)
        rows_for_change.iloc[interval1:interval2,rows_for_change.columns.get_loc('building_number')] = percentage_for_insert.iloc[j,percentage_for_insert.columns.get_loc('building_number')]
        interval1 = interval1 + round(rows_for_change.shape[0]*percentage_for_insert.iloc[j,percentage_for_insert.columns.get_loc('percentage')])
        industrial_without_building[industrial_without_building['department_name'] == industrial_without_building['department_name'].unique()[i]] = rows_for_change
    
for i in range (domestic_without_building['department_name'].unique().shape[0]):
    rows_for_change = domestic_without_building[domestic_without_building['department_name'] == domestic_without_building['department_name'].unique()[i]]
    percentage_for_insert = domestic_div_percentage[domestic_div_percentage['department_name'] == domestic_without_building['department_name'].unique()[i]]
    num_of_different_buildings = len(percentage_for_insert)
    interval1 = 0
    interval2 = 0
    for j in range (num_of_different_buildings):
        interval2 = interval2 + round(rows_for_change.shape[0]*percentage_for_insert.iloc[j,percentage_for_insert.columns.get_loc('percentage')])
        if j == (num_of_different_buildings-1):
            interval2 = len(rows_for_change)
        rows_for_change.iloc[interval1:interval2,rows_for_change.columns.get_loc('building_number')] = percentage_for_insert.iloc[j,percentage_for_insert.columns.get_loc('building_number')]
        interval1 = interval1 + round(rows_for_change.shape[0]*percentage_for_insert.iloc[j,percentage_for_insert.columns.get_loc('percentage')])
        domestic_without_building[domestic_without_building['department_name'] == domestic_without_building['department_name'].unique()[i]] = rows_for_change

In [24]:
other_df = pd.concat([other_without_building, other_with_building])
other_df.sort_values('building_number', axis=0, ascending=False, inplace=True, ignore_index=True)

industrial_df = pd.concat([industrial_without_building, industrial_with_building])
industrial_df.sort_values('building_number', axis=0, ascending=False, inplace=True, ignore_index=True)

domestic_df = pd.concat([domestic_without_building, domestic_with_building])
domestic_df.sort_values('building_number', axis=0, ascending=False, inplace=True, ignore_index=True)

In [25]:
other_exceptions = other_df[pd.isnull(other_df['building_number']) == True]
other_exceptions.sort_values('department_number', axis=0, ascending=False, inplace=True, ignore_index=True)
print(other_exceptions['department_number'].unique())

[6932 6380 6260 6204 5901 5709 5507 5502 5107]


In [26]:
print('Other Exceptions Before Cleaning:',len(other_df['department_number'][pd.isnull(other_df['building_number']) == True]))

other_df = other_df[other_df['department_number'] != 6932]
other_df = other_df[other_df['department_number'] != 6260]
other_df = other_df[other_df['department_number'] != 6204]
other_df = other_df[other_df['department_number'] != 5901]

other_df['building_number'][other_df['department_number'] == 6380] = '39'
other_df['building_number'][other_df['department_number'] == 5709] = '41'
other_df['building_number'][other_df['department_number'] == 5507] = '55'
other_df['building_number'][other_df['department_number'] == 5502] = '63'
other_df['building_number'][other_df['department_number'] == 5107] = '58'

print('Other Exceptions After Cleaning:',len(other_df['department_number'][pd.isnull(other_df['building_number']) == True]))

Other Exceptions Before Cleaning: 22
Other Exceptions After Cleaning: 0


In [27]:
industrial_exceptions = industrial_df[pd.isnull(industrial_df['building_number']) == True]
industrial_exceptions.sort_values('department_number', axis=0, ascending=False, inplace=True, ignore_index=True)
print(industrial_exceptions['department_number'].unique())

[6932 6204 5905 5901 5723 5511 5313]


In [28]:
print('Industrial Exceptions Before Cleaning:',len(industrial_df['department_number'][pd.isnull(industrial_df['building_number']) == True]))

industrial_df = industrial_df[industrial_df['department_number'] != 6932]
industrial_df = industrial_df[industrial_df['department_number'] != 6204]
industrial_df = industrial_df[industrial_df['department_number'] != 5905]
industrial_df = industrial_df[industrial_df['department_number'] != 5901]
industrial_df = industrial_df[industrial_df['department_number'] != 5723]

industrial_df['building_number'][industrial_df['department_number'] == 5511] = '63'
industrial_df['building_number'][industrial_df['department_number'] == 5313] = '72'

print('Industrial Exceptions After Cleaning:',len(industrial_df['department_number'][pd.isnull(industrial_df['building_number']) == True]))

Industrial Exceptions Before Cleaning: 21
Industrial Exceptions After Cleaning: 0


In [29]:
domestic_exceptions = domestic_df[pd.isnull(domestic_df['building_number']) == True]
domestic_exceptions.sort_values('department_number', axis=0, ascending=False, inplace=True, ignore_index=True)
print(domestic_exceptions['department_number'].unique())

[8298 6932 6922 6912 6733 6260 6204 5930 5905 5901 5812 5807 5803 5709 5708 5705 5700 5520 5421 5331 5323 5109 5105 5000 3919 3907]


In [30]:
print('Domestic Exceptions Before Cleaning:',len(domestic_df['department_number'][pd.isnull(domestic_df['building_number']) == True]))

domestic_df = domestic_df[domestic_df['department_number'] != 8298]
domestic_df = domestic_df[domestic_df['department_number'] != 6932]
domestic_df = domestic_df[domestic_df['department_number'] != 6922]
domestic_df = domestic_df[domestic_df['department_number'] != 6912]
domestic_df = domestic_df[domestic_df['department_number'] != 6260]
domestic_df = domestic_df[domestic_df['department_number'] != 6204]
domestic_df = domestic_df[domestic_df['department_number'] != 5930]
domestic_df = domestic_df[domestic_df['department_number'] != 5905]
domestic_df = domestic_df[domestic_df['department_number'] != 5901]
domestic_df = domestic_df[domestic_df['department_number'] != 5807]
domestic_df = domestic_df[domestic_df['department_number'] != 5421]

domestic_df['building_number'][domestic_df['department_number'] == 6733] = '15'
domestic_df['building_number'][domestic_df['department_number'] == 5812] = '34'
domestic_df['building_number'][domestic_df['department_number'] == 5803] = '71'
domestic_df['building_number'][domestic_df['department_number'] == 5709] = '41'
domestic_df['building_number'][domestic_df['department_number'] == 5708] = '34'
domestic_df['building_number'][domestic_df['department_number'] == 5705] = '93A'
domestic_df['building_number'][domestic_df['department_number'] == 5700] = '70'
domestic_df['building_number'][domestic_df['department_number'] == 5520] = '58'
domestic_df['building_number'][domestic_df['department_number'] == 5331] = '72'
domestic_df['building_number'][domestic_df['department_number'] == 5323] = '72'
domestic_df['building_number'][domestic_df['department_number'] == 5109] = '51'
domestic_df['building_number'][domestic_df['department_number'] == 5105] = '29'
domestic_df['building_number'][domestic_df['department_number'] == 5000] = '62'
domestic_df['building_number'][domestic_df['department_number'] == 3919] = '34'
domestic_df['building_number'][domestic_df['department_number'] == 3907] = '30'

print('Domestic Exceptions After Cleaning:',len(domestic_df['department_number'][pd.isnull(domestic_df['building_number']) == True]))

Domestic Exceptions Before Cleaning: 416
Domestic Exceptions After Cleaning: 0


In [31]:
print ('Other Building Numbers:', other_df['building_number'].unique())
print()
other_df = other_df[other_df['building_number'] != 'M8']
other_df = other_df[other_df['building_number'] != 'M6']
other_df.reset_index(drop=True, inplace=True)
other_df.drop(['price','department_number', 'department_name', 'building_name'], axis=1, inplace=True)

print ('Industrial Building Numbers:', industrial_df['building_number'].unique())
print()
industrial_df = industrial_df[industrial_df['building_number'] != 'M9']
industrial_df = industrial_df[industrial_df['building_number'] != 'M7']
industrial_df = industrial_df[industrial_df['building_number'] != 'M6']
industrial_df.reset_index(drop=True, inplace=True)
industrial_df.drop(['price','department_number', 'department_name', 'building_name'], axis=1, inplace=True)

print ('Domestic Building Numbers:', domestic_df['building_number'].unique())
print()
domestic_df = domestic_df[domestic_df['building_number'] != 'M9']
domestic_df = domestic_df[domestic_df['building_number'] != 'M7']
domestic_df = domestic_df[domestic_df['building_number'] != 'M6']
domestic_df.reset_index(drop=True, inplace=True)
domestic_df.drop(['price','department_number', 'department_name', 'building_name'], axis=1, inplace=True)

Other Building Numbers: ['M8' 'M6' '96' '95' '71' '63' '62' '59' '51' '43' '42' '41' '40' '39' '29' '55' '58']

Industrial Building Numbers: ['M9' 'M7' 'M6' '63' '62' '60' '59' '58' '42' '41' '40' '39' '38' '29' '72']

Domestic Building Numbers: ['M9' 'M7' 'M6' '98' '96' '95' '93A' '93' '74' '72' '71' '70' '65' '64' '63' '62' '59' '58' '56' '55' '51' '42' '41' '40' '39' '33' '30' '29' '26' '22' '18' '17' '16'
 '15' '34']



In [32]:
other_df['operating_date'] = pd.to_datetime(other_df['operating_date'])
industrial_df['operating_date'] = pd.to_datetime(industrial_df['operating_date'])
domestic_df['operating_date'] = pd.to_datetime(domestic_df['operating_date'])

In [33]:
months=12
for i in range(1, months+1):

    string1 = '2019-'
    if i != 12:
        string2 = str(i+1)
        final_date = string1+string2
    else:        
        string2 = str(i)
        string3 = '-31'
        final_date = string1+string2+string3
    final_date = pd.to_datetime(final_date)

    other_final = other_df[~(other_df['operating_date'] >= final_date)]
    other_final = other_final.drop(['operating_date'], axis = 1, inplace=False)

    industrial_final = industrial_df[~(industrial_df['operating_date'] >= final_date)]
    industrial_final = industrial_final.drop(['operating_date'], axis = 1, inplace=False)

    domestic_final = domestic_df[~(domestic_df['operating_date'] >= final_date)]
    domestic_final = domestic_final.drop(['operating_date'], axis = 1, inplace=False)

    other_final = other_final.groupby(['building_number']).size().reset_index(name='r_other')
    industrial_final = industrial_final.groupby(['building_number']).size().reset_index(name='r_industrial')
    domestic_final = domestic_final.groupby(['building_number']).size().reset_index(name='r_domestic')

    total_refrigerators = pd.merge(domestic_final, industrial_final, how='outer', on='building_number')
    total_refrigerators = pd.merge(total_refrigerators, other_final, how='outer', on='building_number')

    total_refrigerators = total_refrigerators.fillna(0)
    total_refrigerators.sort_values('building_number', axis=0, ascending=True, inplace=True, ignore_index=True)

    file_name = 'total_refrigerators_'+string1+str(i)+'.csv'
    total_refrigerators.to_csv(file_name, index=False, encoding="utf-8")